# Семинар по рекуррентным нейронным сетям
На этом семинаре мы обучим несколько рекуррентных архитектур для решения задачи сентимент-анализа, то есть предсказания метки тональности предложения.

В общем случае рекуррентная нейронная сеть предназначена для обработки последовательности произвольной длины. Однако при реализации метода оказывается проще зафиксировать длину последовательности (даже в pytorch с их динамическими графами :) Мы так и поступим.

При выполнении задания вы обучите LSTM с разным уровнем "коробочности", а также познакомитесь с различными способами применения дропаута к рекуррентным архитектурам. В рекуррентных архитектурах вариантов, куда можно наложить бинарную маску шума, гораздо больше, чем в нейросетях прямого прохода.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоемкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

### Гиперпараметры

In [1]:
vocab_size = 20000 
index_from = 3
n_hidden = 32 # 128
n_emb = 32 # 128
seq_len = 32 # 200
# small network on small data for seminar purposes
# after # normal size

batch_size = 128
learning_rate = 0.001
num_epochs = 30

### Загрузка данных
Функция load_matrix_imdb скачивает матричные данные, перемешивает и загружает их в numpy-массивы.

Если у вас не установлен wget, скачайте [архив imdb.npz](https://s3.amazonaws.com/text-datasets/imdb.npz)

In [4]:
from rnn_utils import load_matrix_imdb
import numpy as np
import torch
import torch.utils.data

In [6]:
np.random.seed(0)
(X_train, y_train), (X_test, y_test) = load_matrix_imdb(num_words=vocab_size,
                                                        maxlen=seq_len)

In [7]:
set(y_train) # binary classification

{0, 1}

In [8]:
X_train.shape, X_test.shape

((25000, 32), (25000, 32))

In [9]:
X_train[0] # sequence of coded words

array([  1.00000000e+00,   1.40000000e+01,   2.20000000e+01,
         1.60000000e+01,   4.30000000e+01,   5.30000000e+02,
         9.73000000e+02,   1.62200000e+03,   1.38500000e+03,
         6.50000000e+01,   4.58000000e+02,   4.46800000e+03,
         6.60000000e+01,   3.94100000e+03,   4.00000000e+00,
         1.73000000e+02,   3.60000000e+01,   2.56000000e+02,
         5.00000000e+00,   2.50000000e+01,   1.00000000e+02,
         4.30000000e+01,   8.38000000e+02,   1.12000000e+02,
         5.00000000e+01,   6.70000000e+02,   2.00000000e+00,
         9.00000000e+00,   3.50000000e+01,   4.80000000e+02,
         2.84000000e+02,   5.00000000e+00])

In [10]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [11]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long), 
                               torch.tensor(y_train, dtype=torch.long))

In [12]:
test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long), 
                               torch.tensor(y_test, dtype=torch.long))

In [13]:
train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

In [14]:
test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

### Сборка и обучение RNN в pytorch

In [15]:
import os
import torch.optim as optim
import torch.nn as nn

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандарную рекуррентную архитектуру для сентимент-анализа: слой представлений, слой LSTM и полносвязный слой, предсказывающий выход по последнему скрытому состоянию.

Ниже дан код для сборки и обучения нашей нейросети. Обратите внимание на ### pay attention here, указывающие на особенности кода при использовании рекуррентных слоев. 

Чтобы потом было удобно проводить сравнение времени работы разных моделей, запустите код ниже и измерьте время обучения на вашей системе.

In [17]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, rec_layer=nn.LSTM, embedding=nn.Embedding, \
                 dropout=None):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = embedding(vocab_size, embedding_dim)
        if dropout:
            self.rnn = rec_layer(embedding_dim, hidden_dim, dropout=dropout)
        else:
            self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn(embedding) # pay attention here!
        res = self.hidden2label(out[-1])
        return torch.sigmoid(res)
    

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

In [18]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=nn.LSTM,
                       dropout=None).to(device)

In [19]:
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

In [20]:
def train_epoch(train_loader, model, lossfun, optimizer, device):
    model.train()
    for it, traindata in enumerate(train_loader):
        train_inputs, train_labels = traindata
        train_inputs = train_inputs.to(device) 
        train_labels = train_labels.to(device)
        train_labels = torch.squeeze(train_labels)

        model.zero_grad()        
        output = model(train_inputs.t()) # pay attention here!

        loss = lossfun(output.view(-1), train_labels.float())
        loss.backward()
        optimizer.step()

def evaluate(loader, model, lossfun, device):
    model.eval()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    for it, data in enumerate(loader):
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        output = model(inputs.t()) # pay attention here!
        loss = lossfun(output.view(-1), labels.float())
        total_loss += loss.item()

        # calc testing acc        
        pred = output.view(-1) > 0.5
        correct = pred == labels.byte()
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total
    

def train(train_loader, test_loader, model, lossfun, optimizer, \
          device, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    for epoch in range(num_epochs):
        train_epoch(train_loader, model, lossfun, optimizer, device)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, device)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, device)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [21]:
%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 81.943, Testing Loss: 84.106, Training Acc: 0.640, Testing Acc: 0.609
Epoch:   2/ 30 Training Loss: 74.694, Testing Loss: 79.745, Training Acc: 0.702, Testing Acc: 0.651
Epoch:   3/ 30 Training Loss: 68.774, Testing Loss: 77.482, Training Acc: 0.731, Testing Acc: 0.674
Epoch:   4/ 30 Training Loss: 62.600, Testing Loss: 74.766, Training Acc: 0.768, Testing Acc: 0.689
Epoch:   5/ 30 Training Loss: 58.975, Testing Loss: 74.408, Training Acc: 0.787, Testing Acc: 0.698
Epoch:   6/ 30 Training Loss: 54.464, Testing Loss: 75.640, Training Acc: 0.806, Testing Acc: 0.706
Epoch:   7/ 30 Training Loss: 51.978, Testing Loss: 73.408, Training Acc: 0.821, Testing Acc: 0.705
Epoch:   8/ 30 Training Loss: 49.261, Testing Loss: 77.581, Training Acc: 0.828, Testing Acc: 0.709
Epoch:   9/ 30 Training Loss: 44.459, Testing Loss: 80.636, Training Acc: 0.850, Testing Acc: 0.712
Epoch:  10/ 30 Training Loss: 42.276, Testing Loss: 76.176, Training Acc: 0.865, Testing Acc: 0.711


Нерегуляризованные LSTM часто быстро переобучаются (и мы это видим по точности на контроле). Чтобы с этим бороться, часто используют L2-регуляризацию и дропаут.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. По [ссылке](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) доступен хороший обзор дропаутов для RNN.

Мы реализуем два варианта дропаута для RNN (и третий дополнительно). Заодно увидим, что для реализации различных усовершенствований рекуррентной архитектуры приходится "вскрывать" слой до различной "глубины".

### Реализация дропаута по статье Гала и Гарамани
Начнем с дропаута, описанного в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется перейти от использования слоя nn.LSTM, полностью скрывающего от нас рекуррентную логику, к использованию слоя nn.LSTMCell, обрабатывающего лишь один временной шаг нашей последовательности (а всю логику вокруг придется реализовать самостоятельно). 

Допишите класс RNNLayer. При dropout=0 ваш класс должен работать как обычный слой LSTM, а при dropout > 0 накладывать бинарную маску на входной и скрытый вектор на каждом временном шаге, причем эта маска должна быть одинаковой во все моменты времени.

Дропаут Гала и Гарамани в виде формул (m обознаает маску дропаута):
$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$
(далее обычный шаг рекуррентной архитектуры, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$

In [42]:
def init_h0_c0(num_objects, hidden_size, some_existing_tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    ### your code here
    h0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    c0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    return h0, c0

In [68]:
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)
    
    return dropout masks of size input_size, hidden_size if p is not None
    return one masks if p is None
    """
    ### your code here
    mask0 = some_existing_tensor.new_ones(input_size)
    mask1 = some_existing_tensor.new_ones(hidden_size)
    if p:
        if is_training:
            mask0 *= p
            mask0 = torch.bernoulli_(mask0)
            mask1 *= p
            mask1 = torch.bernoulli_(mask1)
        else:
            mask0 *= (1-p)
            #mask0 = torch.bernoulli_(mask0)
            mask1 *= (1-p)
            #mask1 = torch.bernoulli_(mask1)
    return mask0, mask1

In [54]:
class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=None):
        super(RNNLayer, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.dropout = dropout
        self.rnn_cell = nn.LSTMCell(input_size, hidden_size)
        
    def forward(self, inp):
        # initialize h_0, c_0
        h_0, c_0 = init_h0_c0(inp.shape[1], self.hidden_size, inp)
        
        # gen masks
        input_mask, hidden_mask = gen_dropout_mask(self.input_size, \
                                                   self.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   inp)
        
        
        ### your code here
        ### implement recurrent logic and return what nn.LSTM returns
        ### do not forget to apply generated dropout masks!
        out = []
        h, c = h_0, c_0
        for i in range(inp.shape[0]):
            h, c = self.rnn_cell(inp[i]*input_mask, (h*hidden_mask, c))
            out.append(h)
        return out, (h, c)

Протестируйте реализованную модель с выключенным дропаутом (слой RNNLayer надо передать в RNNClassifier в качестве rec_layer). Замерьте время обучения (%time). Сильно ли оно увеличилось по сравнению с nn.LSTM (LSTM "из коробки")?

In [52]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=None).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 83.817, Testing Loss: 85.211, Training Acc: 0.619, Testing Acc: 0.597
Epoch:   2/ 30 Training Loss: 75.802, Testing Loss: 79.930, Training Acc: 0.689, Testing Acc: 0.651
Epoch:   3/ 30 Training Loss: 69.843, Testing Loss: 77.057, Training Acc: 0.730, Testing Acc: 0.669
Epoch:   4/ 30 Training Loss: 64.326, Testing Loss: 74.675, Training Acc: 0.762, Testing Acc: 0.687
Epoch:   5/ 30 Training Loss: 59.394, Testing Loss: 73.797, Training Acc: 0.784, Testing Acc: 0.698
Epoch:   6/ 30 Training Loss: 56.637, Testing Loss: 75.094, Training Acc: 0.790, Testing Acc: 0.698
Epoch:   7/ 30 Training Loss: 51.663, Testing Loss: 75.264, Training Acc: 0.819, Testing Acc: 0.708
Epoch:   8/ 30 Training Loss: 48.004, Testing Loss: 76.165, Training Acc: 0.835, Testing Acc: 0.709
Epoch:   9/ 30 Training Loss: 45.135, Testing Loss: 77.664, Training Acc: 0.849, Testing Acc: 0.710
Epoch:  10/ 30 Training Loss: 44.126, Testing Loss: 78.651, Training Acc: 0.854, Testing Acc: 0.699


Протестируйте полученную модель c dropout=0.5, вновь замерив время обучения. Получилось ли побороть переобучение? Сильно ли дольше обучается данная модель по сравнению с предыдущей? (доп. время тратится на генерацию масок дропаута).

In [71]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=0.5).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 88.140, Testing Loss: 88.214, Training Acc: 0.533, Testing Acc: 0.527
Epoch:   2/ 30 Training Loss: 87.613, Testing Loss: 87.859, Training Acc: 0.555, Testing Acc: 0.541
Epoch:   3/ 30 Training Loss: 85.724, Testing Loss: 86.419, Training Acc: 0.600, Testing Acc: 0.577
Epoch:   4/ 30 Training Loss: 83.533, Testing Loss: 84.832, Training Acc: 0.630, Testing Acc: 0.604
Epoch:   5/ 30 Training Loss: 82.476, Testing Loss: 84.109, Training Acc: 0.645, Testing Acc: 0.614
Epoch:   6/ 30 Training Loss: 80.211, Testing Loss: 82.637, Training Acc: 0.662, Testing Acc: 0.630
Epoch:   7/ 30 Training Loss: 78.882, Testing Loss: 81.773, Training Acc: 0.674, Testing Acc: 0.637
Epoch:   8/ 30 Training Loss: 77.066, Testing Loss: 80.740, Training Acc: 0.690, Testing Acc: 0.648
Epoch:   9/ 30 Training Loss: 75.296, Testing Loss: 79.787, Training Acc: 0.703, Testing Acc: 0.657
Epoch:  10/ 30 Training Loss: 74.299, Testing Loss: 79.022, Training Acc: 0.708, Testing Acc: 0.663


### Реализация дропаута по статье Гала и Гарамани. Дубль 2

<начало взлома pytorch>

При разворачивании цикла по времени средствами python обучение рекуррентной нейросети сильно замедляется. Однако для реализации дропаута Гала и Гарамани необязательно явно задавать в коде домножение нейронов на маски. Можно схитрить и обойтись использованием слоя nn.LSTM: перед вызовом forward слоя nn.LSTM подменять его веса на веса, домноженные по строкам на маски. А обучаемые веса хранить отдельно. Именно так этот дропаут реализован в библиотеке fastai, код из которой использован в ячейке ниже.

Такой слой реализуется в виде обертки над nn.LSTM. Допишите класс:

In [53]:
import warnings

In [65]:
class FastRNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(FastRNNLayer, self).__init__()
        self.module = nn.LSTM(input_size, hidden_size)
        self.dropout = dropout
        self.layer_names = ['weight_hh_l0', 'weight_ih_l0']
        for layer in self.layer_names:
            # Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            
    def _setweights(self):
        "Apply dropout to the raw weights."
        ### your code here
        ### generate input_mask and hidden_mask (use function gen_dropout_mask)
        input_mask, hidden_mask = gen_dropout_mask(self.module.input_size, \
                                                   self.module.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   self.module.weight_hh_l0.data)
        for layer, mask in zip(self.layer_names, (hidden_mask, input_mask)):
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = raw_w * mask

    def forward(self, *args):
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            
            ### your code here
            ### set new weights of self.module and call its forward
            self._setweights()
            return self.module.forward(*args)

    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

Протестируйте полученный слой (вновь подставив его в RNNClassifier в качестве rec_layer) с dropout=0.5. Сравните время обучения с предыдущими моделями. Проследите, чтобы качество получилось такое же, как при первой реализации этого дропаута.

In [70]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=FastRNNLayer,
                       dropout=0.5).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 88.126, Testing Loss: 88.234, Training Acc: 0.533, Testing Acc: 0.523
Epoch:   2/ 30 Training Loss: 87.568, Testing Loss: 87.864, Training Acc: 0.554, Testing Acc: 0.542
Epoch:   3/ 30 Training Loss: 85.890, Testing Loss: 86.674, Training Acc: 0.594, Testing Acc: 0.572
Epoch:   4/ 30 Training Loss: 84.923, Testing Loss: 85.988, Training Acc: 0.614, Testing Acc: 0.586
Epoch:   5/ 30 Training Loss: 82.680, Testing Loss: 84.413, Training Acc: 0.639, Testing Acc: 0.604
Epoch:   6/ 30 Training Loss: 80.518, Testing Loss: 83.125, Training Acc: 0.655, Testing Acc: 0.620
Epoch:   7/ 30 Training Loss: 78.116, Testing Loss: 81.429, Training Acc: 0.676, Testing Acc: 0.638
Epoch:   8/ 30 Training Loss: 76.502, Testing Loss: 80.371, Training Acc: 0.691, Testing Acc: 0.649
Epoch:   9/ 30 Training Loss: 74.534, Testing Loss: 79.262, Training Acc: 0.705, Testing Acc: 0.657
Epoch:  10/ 30 Training Loss: 73.071, Testing Loss: 78.453, Training Acc: 0.714, Testing Acc: 0.663


</конец взлома pytorch>

### Реализация дропаута по статье Семениуты и др
Перейдем к реализации дропаута для LSTM по статье [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165). 

Этот метод применения дропаута не менее популярен, чем предыдущий. Его особенность состоит в том, что он придуман специально для гейтовых архитектур. В контексте LSTM этот дропаут накладывается только на информационный поток (m_h - маска дропаута):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
На входы $x_t$ маска накладывается как в предыдущем дропауте. Впрочем, на входы маску можно наложить вообще до вызова рекуррентного слоя.

Согласно статье, маска дропаута может быть как одинаковая, так и разная для всех моментов времени. Мы сделаем одинаковую.

Для реализации этого дропаута можно: а) самостоятельно реализовать LSTM (интерфейса LSTMCell не хватит) б) снова воспользоваться трюком с установкой весов (но тут мы опираемся на свойство hanh(0)=0, к тому же, трюк в данном случае выглядит менее тривиально, чем с дропаутом Гала). Предлагается реализовать дрпоаут по сценираю а. Допишите класс:

In [ ]:
class HandmadeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.):
        super(HandmadeLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.input_weights = nn.Linear(input_size, 4 * hidden_size)
        self.hidden_weights = nn.Linear(hidden_size, 4 * hidden_size)
        
        self.reset_params()


    def reset_params(self):
        """
        initialization as in Pytorch
        do not forget to call this method!
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(0, stdv)
            

    def forward(self, inp, hidden=None):
        ### your code here
        # use functions init_h0_c0 and gen_dropout_masks defined above

Протестируйте вашу реализацию без дропаута (проконтролируйте качество и сравните время обучения с временем обучения nn.LSTM и RNNLayer), а также с dropout=0.5. Сравните качество модели с таким дропаутом с качеством модели с дропаутом Гала и Гарамани.

### Zoneout
Это еще одна модификация идеи дропаута применительно к рекуррентным нейросетям. В Zoneout на каждом временном шаге с вероятностью p компонента скрытого состояния обновляется, а с вероятностью 1-p берется с предыдущего шага. 
В Виде формул (m^t_h - бинарная маска):
 
(сначала обычный рекуррентный переход, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
Затем Zoneout:
$$
h_t = h_t * m_h^t + h_{t-1}*(1-m_h^t)
$$
В этом методе маска уже должна быть разная во все моменты времени (иначе метод упрощается до дропаута Гала и Гарамани). На входы $x_t$ вновь можно накладывать маску до начала работы рекуррентного слоя.  

Если у вас осталось время, вы можете реализовать этот метод. Выберите основу из трех рассмотренных случаев самостоятельно.
